In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import io
import json
import pytz
from IPython.core.display import HTML

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.7607.0,0.5.2343,"{'relation': 'RequestLogs', 'href': 'http://om..."


# Read in and format data

In [2]:
scope = "srs"
df = pd.read_csv("Data.csv")
df["Date"] = pd.to_datetime(df["Date"], utc=True)
df

,Port,Currency,Date,Sys,Value,Class
0,PortA,AUD,2021-07-13 00:00:00+00:00,FSC,50000000,C
1,PortA,AUD,2021-07-13 00:00:00+00:00,FSC,400000,C
2,PortA,CAD,2021-07-13 00:00:00+00:00,FSC,138234286.2,C
3,PortA,CAD,2021-07-13 00:00:00+00:00,FSC,492384234,C
4,PortA,CHF,2021-07-13 00:00:00+00:00,FSC,203402,C
5,PortA,EUR,2021-07-13 00:00:00+00:00,FSC,2-35402950,C
6,PortA,EUR,2021-07-13 00:00:00+00:00,FSC,24502394,C
7,PortA,EUR,2021-07-15 00:00:00+00:00,FTC,22342054,C
8,PortA,EUR,2021-07-27 00:00:00+00:00,FTC,2349130942,C
9,PortA,EUR,2021-08-12 00:00:00+00:00,FTC,23489234,C


## Define data ID

In [3]:
srs_api = api_factory.build(lu.StructuredResultDataApi)

srs_ids = [
    lm.StructuredResultDataId(
        source="Client", 
        code='SRS_File', 
        effective_at=datetime(2021, 1, 27, tzinfo=timezone.utc),
        result_type="Raw"
    )
]

## Upsert Structured Data

In [4]:
version= "1.01"

with open("./Data.csv", 'r') as myfile: 
    data = myfile.read()

request_body={
    "key1": lm.UpsertStructuredResultDataRequest(
    id=srs_ids[0],
    data=lm.StructuredResultData(
    document_format="csv",
    version=version,
    name="Data file",
    document=data
        )
    )
}


result = srs_api.upsert_structured_result_data(
    scope=scope,
    request_body=request_body)
    

## Return Data from store

In [5]:
navs = srs_api.get_structured_result_data(
        scope=scope, 
        request_body={
            "key1": srs_ids[0]
        }
    )
    
s = io.StringIO(navs.values["key1"].document)
    
nav_doc = pd.read_csv(s)

In [6]:
nav_doc

,Port,Currency,Date,Sys,Value,Class
0,PortA,AUD,20210713 00:00:00,FSC,50000000,C
1,PortA,AUD,20210713 00:00:00,FSC,400000,C
2,PortA,CAD,20210713 00:00:00,FSC,138234286.2,C
3,PortA,CAD,20210713 00:00:00,FSC,492384234,C
4,PortA,CHF,20210713 00:00:00,FSC,203402,C
5,PortA,EUR,20210713 00:00:00,FSC,2-35402950,C
6,PortA,EUR,20210713 00:00:00,FSC,24502394,C
7,PortA,EUR,20210715 00:00:00,FTC,22342054,C
8,PortA,EUR,20210727 00:00:00,FTC,2349130942,C
9,PortA,EUR,20210812 00:00:00,FTC,23489234,C
